In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data=pd.read_csv("../../../Dataset/credit_default_train.csv")

In [12]:
#FORMULA 1
#Se pa è maggiore di ba allora ps deve essere -1 (tutto pagato)
count=0
rows=set()
tot=0
for i in range(0, len(data)):
    for j in range(12, 17):
        ba=data.iat[i, j]
        if(ba>0):
            pa=data.iat[i, j+5]
            if((ba-pa)<=0):
                ps=data.iat[i, j-6]
                tot+=1
                if(ps!=-1):
                    count+=1
                    rows.add(i)
print("Total number of cells: ", tot)
print("Total number of errors: ", count)
print('Rows involved: ', len(rows))
perc=(count*100)/tot
print("Percentage of error: ", perc)

count  2488
Rows:  841
Tot:  10913
Percentage of error:  22.798497205168147


In [124]:
#combinazioni controllate dalla FORMULA 1
for j in range(12, 17):
    print("ROW")
    print(data.columns[j])
    print(data.columns[j+5])
    print(data.columns[j-6])

ROW
ba-aug
pa-sep
ps-aug
ROW
ba-jul
pa-aug
ps-jul
ROW
ba-jun
pa-jul
ps-jun
ROW
ba-may
pa-jun
ps-may
ROW
ba-apr
pa-may
ps-apr


In [132]:
#Correzione FORMULA 1
#Sistemo valori -1
for i in range(0, len(data)):
    for j in range(12, 17):
        ba=data.iat[i, j]
        if(ba>0):
            pa=data.iat[i, j+5]
            if((ba-pa)<=0):
                ps=data.iat[i, j-6]
                if(ps!=-1):
                    data.iloc[i,j-6]=-1

In [13]:
#FORMULA 2
#Se BA==0 non è detto che PS debba essere -2. 
#Perchè il BA==0 può essere conseguenza di un precedente BA<0 che è stato conpensato da una nuova spesa (portandolo a -2)
tot=0
count=0
rows=set()
for i in range(0, len(data)):
    for j in range(11,16):
        ba=data.iat[i,j]
        precBa=data.iat[i,j+1]
        pa=data.iat[i,j+6]
        ps=data.iat[i,j-6]
        if(ba<=0):
            if(ba==(precBa-pa)):
                tot+=1
                if(ps!=-2):
                    count+=1
                    rows.add(i)
print("Total number of cells: ", tot)
print("Total number of errors: ", count)
print('Rows involved: ', len(rows))
perc=(count*100)/tot
print("Percentage of error: ", perc)

Different from -2:  1814
Rows:  1585
Tot:  5510
Percentage of error:  32.92196007259528


In [ ]:
#combinazioni controllate dalla FORMULA 2
for j in range(11,16):
    print("ROW")
    print(data.columns[j])
    print(data.columns[j+1])
    print(data.columns[j+6])
    print(data.columns[j-6])

In [134]:
#Correzione FORMULA 2
#sistemo valori -2
for i in range(0, len(data)):
    for j in range(11,16):
        ba=data.iat[i,j]
        precBa=data.iat[i,j+1]
        pa=data.iat[i,j+6]
        ps=data.iat[i,j-6]
        if(ba<=0):
            if(ba==(precBa-pa)):
                if(ps!=-2):
                    data.iloc[i,j-6]=-2

In [14]:
#FORMULA 3
#se ps==0 e ba>0 allora pa>0
#non verificabile perchè non si sa qual è il credito minimo da pagare
rows=set()
count=0
tot=0
for i in range(0,len(data)):
    for j in range(6,11):
        ps=data.iat[i,j]
        ba=data.iat[i,j+6]
        if((ps==0) & (ba>0)):
            pa=data.iat[i,j+6+5]
            tot+=1
            if(pa<=0):
                count+=1
                rows.add(i)
print("Total number of cells: ", tot)
print("Total number of errors: ", count)
print('Rows involved: ', len(rows))
perc=(count*100)/tot
print("Percentage of error: ", perc)

Count 655
Rows 545
Tot:  26410
Percentage of error:  2.4801211662249147


In [ ]:
#combinazioni controllate dalla FORMULA 3
for j in range(6,11):
    print("ROW")
    print(data.columns[j])
    print(data.columns[j+6])
    print(data.columns[j+6+5])

In [15]:
#FORMULA 4
#se ps>=0 e ba>0 allora il successivo ba deve essere maggiore di (ba-pa) precedenti
rows=set()
count=0
tot=0
for i in range(0,len(data)):
    for j in range(5,10):
        ps=data.iat[i,j]
        ba=data.iat[i,j+6]
        if((ps>=0) & (ba>0)):
            baPrec=data.iat[i,j+6+1]
            paPrec=data.iat[i,j+6+1+5]
            dif=baPrec-paPrec
            tot+=1
            if(ba<=dif):
                count+=1
                rows.add(i)
print("Total number of cells: ", tot)
print("Total number of errors: ", count)
print('Rows involved: ', len(rows))
perc=(count*100)/tot
print("Percentage of error: ", perc)

Count 738
Rows 431
Tot:  32670
Percentage of error:  2.2589531680440773


In [89]:
#combinazioni controllate dalla FORMULA 4
for j in range(5,10):
    print("ROW")
    print(data.columns[j])
    print(data.columns[j+6])
    print(data.columns[j+6+1])
    print(data.columns[j+6+1+5])

ROW
ps-sep
ba-sep
ba-aug
pa-sep
ROW
ps-aug
ba-aug
ba-jul
pa-aug
ROW
ps-jul
ba-jul
ba-jun
pa-jul
ROW
ps-jun
ba-jun
ba-may
pa-jun
ROW
ps-may
ba-may
ba-apr
pa-may


In [137]:
#FORMULA 5
#se ho un ba<0 allora deve essere >= della differenza tra ba e pa precedenti
count=0
rows=set()
for i in range(0, len(data)):
    for j in range(11,16):
        ba=data.iat[i,j]
        if(ba<0):
            precBa=data.iat[i,j+1]
            pa=data.iat[i,j+6]
            diff=ba-pa
            if(ba<diff):
                count+=1
                rows.add(i)
print("Count",count)
print("Rows", len(rows))

Count 0
Rows 0
